In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import tabula
import re
import PyPDF2
import locale
import io
import glob
import sys
import gc


os.environ['w2n.lang'] = 'es'
locale.setlocale(locale.LC_TIME, 'es_ES')

def fill_text_field(driver, xpath, text):
    # Funcion que llena texto en un campo de la pagina
    text_input = driver.find_element(By.XPATH, xpath)
    text_input.clear()
    text_input.send_keys(text)

def select_option(driver, xpath, index):
    # Selecciona una opcion en un menu desplegable
    select_object = Select(driver.find_element(By.XPATH, xpath))
    select_object.select_by_value(index)


url = 'https://www.pjud.cl/tribunales/corte-suprema'
chromeOptions = webdriver.ChromeOptions()
path = os.path.join(os.getcwd(), "output\\")
prefs = {"download.default_directory" : path,  "directory_upgrade": True}
chromeOptions.add_experimental_option("prefs",prefs)



driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get(url)
#NOTA: al abrirse el driver de Chrome, seleccionar Configuración-Descargar PDFs

driver.implicitly_wait(10)


<ipython-input-5-7a82b02f9201>:43: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)


In [6]:

años = list(range(2015,2018))

meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for año in años:
    # Estos indices se obtienen de las opciones de la pagina, cada uno es un mes, cambiar para obtener mas meses
    #for index in range(1, 2):
        # Se envia la info a la pagina
    select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[1]/div/select', str(año))
    for mes in meses:
        select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[2]/div/select', mes)
        select = Select(driver.find_element(By.XPATH, "/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select")) #get all the options into a list
        optionsList = []
        for item in select.options:
            optionsList.append(item.get_attribute("value"))            
            for optionValue in optionsList:
                select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select', optionValue)

        # Se hace click con los datos llenados
                driver.find_element(By.XPATH, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[4]/button').click()
        
        # Se descargan los datos
                wait = WebDriverWait(driver, 10) 
    #descarga por salas cambia último tr: 1, 2, 3, 4 - no siempre está disponible, el mismo día, las mismas salas
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[1]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[2]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[3]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[4]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
driver.close




NameError: name 'text' is not defined

In [9]:
columns = ['Fecha', 'Sala', 'Integrantes', 'Causa']
df = pd.DataFrame(columns = columns)

palabras = ['licencia', 'comisi ón', 'permiso', 'feriado', 'inhabilidad', 'subroga', 'vacancia']






pdf_files = glob.glob(os.path.join("C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/output",'*.pdf'))

count = 0
extractedtext = ""
for pdf_file in pdf_files:
    print(pdf_file)
    pdfFileObj = open(pdf_file,'rb')               
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   
    num_pages = pdfReader.numPages
    pdfPageObj = pdfReader.getPage(0)
    text = pdfPageObj.extractText()
    Causa = []
    if re.search("INSTALACI", text):
        Integrantes = re.findall('(?<=ÑOR).*?(?=\s)', text)
        try:
            Fecha = re.findall('(?<=FIRMADIGITAL).*?(?=\\\n)', text)[0]
        except:
            pass
        try:
            Sala = re.findall('(?<=\\s).*?(?=\\sSALA)', text)[0]
        except:
            pass
        for palabra in palabras:         
            if palabra in text:
                Causa.append(palabra)
        df = df.append(
               {"Fecha" : str(Fecha), "Sala" : str(Sala), "Integrantes" : Integrantes, 'Causa' : Causa},
               ignore_index=True,
            )


df['Causa'].head(50)

#Para separar año y día y convertir a número - lleva a formato fecha. Ojo: muy largo y pesado de ejecutar
df['Fecha'] = df['Fecha'].str.replace("treinta y uno", "treintaiuno")
df['Dia'] = df['Fecha'].str.split(' ').str[0] 
df["Ano"] = df["Fecha"].str.extract(r'(\w+)$')
df['Dia'] = df['Dia'].str.replace("treintaiuno", "treinta y uno")

#df['Dia'] = df.Dia.apply(w2n.word_to_num)
df['Dia']

nums = {'^uno$': '1', '^dos$': '2', '^tres$': '3', "^cuatro$":'4', "^cinco$" : '5', "^seis$":'6', "^siete$": '7', 
        '^ocho$':'8',
       '^nueve$':'9', 'diez':'10', 'once':'11', 'doce':'12', 'trece':'13','catorce':'14', 'quince': '15', 'dieciséis':'16',
        'diecisiete':'17', 'dieciocho':'18', 'diecinueve':'19', 'veinte':'20', 'veintiuno':'21','veintidós':'22', 
        'veintitrés':'23', 'veinticuatro':'24', 'veinticinco':'25', 'veintiséis':'26', 'veintisiete':'27',
        'veintiocho':'28', 'veintinueve': '29', '^treinta$': '30', 'treinta y uno': '31'}
        

for old, new in nums.items():
    df['Dia'] = df['Dia'].str.replace(old, new, regex=True)

df['Dia'].value_counts()

nums = {'^veinte$': '20', '^veintiuno$': '21', '^veintidós$': '22'}
        

for old, new in nums.items():
    df['Ano'] = df['Ano'].str.replace(old, new, regex=True)

df['Ano'].value_counts()

#convierte a string para juntar
df['Ano'] = df['Ano'].astype(str)
df['Dia'] = df['Dia'].astype(str)#aisla y convierte a una columna mes
df["Mes"] = df["Fecha"]
df['Mes'] = df['Mes'].str.split(n=1).str[1]
df['Mes'] = df['Mes'].str.lstrip()
df['Mes'] = df['Mes'].str.split(' ').str[1] 

#concatenación fecha en formato
df['Fecha2'] = df['Dia'] + '/' + df['Mes'] + '/' + df['Ano']
#transformación a datetime
df['Fecha2'] = pd.to_datetime(df["Fecha2"], format = '%d/%B/%y')

df2 = df
df2['Presidente'] = df2['Integrantes'].str[0]
df2['Integrante1'] = df2['Integrantes'].str[1]
df2['Integrante2'] = df2['Integrantes'].str[2]
df2['Integrante3'] = df2['Integrantes'].str[3]
df2['Integrante4'] = df2['Integrantes'].str[4]

df3 = df2
df3 = df3.drop(['Fecha', 'Integrantes', 'Dia', 'Ano', 'Mes'], axis = 1)


df3['Causa1'] = df3['Causa'].str[0]
df3['Causa2'] = df3['Causa'].str[1]
df3['Causa3'] = df3['Causa'].str[2]
df3['Causa4'] = df3['Causa'].str[3]


df3 = df3.drop(['Causa'], axis = 1)

df3.drop_duplicates()

df3.to_csv("Scan2.csv")

C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (1).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (10).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (100).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (11).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (12).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (13).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (14).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (15).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (16).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (17).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (18).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (19).pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 (2).pdf
C:/Users/jwil

C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210158.726.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210201.868.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210205.334.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210244.717.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210330.740.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210336.471.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210435.903.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210440.638.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210450.129.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T210530.635.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/produ

C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214142.056.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214203.362.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214224.615.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214245.874.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214328.504.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214349.800.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214411.101.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214432.386.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214515.120.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T214536.416.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/produ

C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222503.861.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222505.832.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222510.919.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222513.635.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222517.020.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222520.366.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222524.244.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222644.650.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222724.812.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T222729.641.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/produ

C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225316.225.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225319.700.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225322.241.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225401.360.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225441.552.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225445.010.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225448.409.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225452.082.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225530.071.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+1 - 2022-08-25T225534.443.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/produ

...

C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202537.054.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202608.933.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202703.338.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202743.675.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202815.655.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202840.355.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202912.304.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T202944.432.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T203015.917.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-0

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T203759.311.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T203845.133.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T203937.503.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T203955.998.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T204027.878.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T204054.259.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T204125.983.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T204214.047.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T204239.548.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-0

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T205716.493.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T205732.187.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T205804.338.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T205825.693.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T205857.506.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T210015.346.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T210031.848.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T210103.917.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-08-26T210159.088.pdf
C:/Users/jwile/WebScrapingMDS/WebScrapingMDS/producto7/scan\Sala+de+Pleno - 2022-0

In [10]:
df3

,Sala,Fecha2,Presidente,Integrante1,Integrante2,Integrante3,Integrante4,Causa1,Causa2,Causa3,Causa4
